## Notes

This was developed to run using FEniCS version 2019.1.0

I ran this code using a docker container, specifically from quay.io/fenicsproject/stable:latest

In [ ]:
from dolfin import *
import numpy as np
#from scipy.integrate import solve_ivp # urgh... the container scipy version is ancient
from scipy.integrate import ode # this will do
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Define parameters
W = 4.0
H = 2.0
sp = 0.4 # a "squeeze" parameter
ny = 64
degree = 2
P = 4.0 # default guess
if degree==2 and ny==64 and W==4 and H==2:
    P = 4.119237152753521 # <-- tuned for W=4,H=2,sp=0.4,ny=64 and 2nd order elements
if degree==2 and ny==128 and W==4 and H==2:
    P = 4.11932766796 # <-- tuned for W=4,H=2,sp=0.4,ny=128 and 2nd order elements
if degree==3 and ny==64 and W==4 and H==2:
    P = 4.11922475139 # <-- tuned for W=4,H=2,sp=0.4,ny=64 and 3rd order elements
if degree==3 and ny==128 and W==4 and H==2:
    P = 4.11931644914 # <-- tuned for W=4,H=2,sp=0.4,ny=128 and 3rd order elements
print("Pressure gradient:",P)

# Setup mesh (change n,m to change the mesh resolution)
nx = int(W/H*ny)
rect_mesh = RectangleMesh(Point(-0.5*W,-0.5*H),Point(0.5*W,0.5*H),nx,ny,"crossed")

# Modify the mesh
mesh = Mesh()
mesh_editor = MeshEditor()
mesh_editor.open(mesh,"triangle",2,2,1)
mesh_editor.init_vertices(rect_mesh.num_vertices())
mesh_editor.init_cells(rect_mesh.num_cells())
used_verts = np.zeros(rect_mesh.num_vertices(),dtype=np.uint8)
for ci in range(rect_mesh.num_cells()):
    c = Cell(rect_mesh,ci)
    vs = []
    for v in vertices(c):
        if used_verts[v.index()]==0:
            xo = v.x(0)
            yo = v.x(1)
            yn = yo*(1.0-sp*np.cos(xo*np.pi/W)**2)
            mesh_editor.add_vertex(v.index(),Point(xo,yn)) # y coordinate is unchanged
            used_verts[v.index()] += 1
        vs.append(v.index())
    mesh_editor.add_cell(ci,np.array(vs,dtype=np.uintp))
mesh_editor.close()

# Now setup and solve the flow velocity
V = FunctionSpace(mesh,"Lagrange",degree)
Vplus = FunctionSpace(mesh,"Lagrange",degree+1)
Vminus = FunctionSpace(mesh,"Lagrange",degree-1)
u = TrialFunction(V)
v = TestFunction(V)
a = inner(grad(u),grad(v))*dx
L = inner(Constant(P),v)*dx
bc = DirichletBC(V,0,"on_boundary")
bcs = [bc]
u = Function(V)
solve(a==L,u,bcs)
print("u inf norm:",np.max(u.vector()[:]))
print("(Corrected pressure:",P/np.max(u.vector()[:]),")")
dudx = project(u.dx(0),V) # Would Vminus be better???
dudy = project(u.dx(1),V)

# Optionally plot the solution
if True:
    plt.figure(figsize=(9,4))
    cf = plot(u)
    plt.colorbar(cf)
    plt.show()

# Define some curves to plot the outline
xs = np.linspace(-W/2,W/2,nx+1)
outline = [np.concatenate([xs,xs[::-1],[-0.5*W]]),
           np.concatenate([+0.5*H*(1-sp*np.cos(xs*np.pi/W)**2),
                           -0.5*H*(1-sp*np.cos(xs[::-1]*np.pi/W)**2),
                           [0.5*H]])
          ]
# Optionally test it out...
if False:
    plt.plot(outline[0],outline[1],'r-',lw=1)
    plt.gca().set_aspect(1.0)
    plt.show()
    
# Produce a meshgrid sampling
ys = np.linspace(-H/2,H/2,ny+1)
Xs,Ys = np.meshgrid(xs,ys)
Us = np.empty_like(Xs)
Us_dx = np.empty_like(Xs)
Us_dy = np.empty_like(Xs)
for i in range(nx+1):
    x = xs[i]
    maxy = 0.5*H*(1-sp*np.cos(x*np.pi/W)**2)
    for j in range(ny+1):
        y = ys[j]
        if abs(y)>maxy:
            Us[j,i] = np.nan
            Us_dx[j,i] = np.nan
            Us_dy[j,i] = np.nan
        else:
            Us[j,i] = u(x,y)
            Us_dx[j,i] = dudx(x,y)
            Us_dy[j,i] = dudy(x,y)
            
# Optionally plot the meshgrid sampling
if True:
    plt.contourf(Xs,Ys,Us,17)
    plt.colorbar()
    plt.plot(outline[0],outline[1],'r-',lw=1)
    plt.gca().set_aspect(1.0)
    plt.show()

In [ ]:
# Optionally, solve once more on a higher degree space and compare...
if False:
    u_ = TrialFunction(Vplus)
    v_ = TestFunction(Vplus)
    a_ = inner(grad(u_),grad(v_))*dx
    L_ = inner(Constant(P),v_)*dx
    bc_ = DirichletBC(Vplus,0,"on_boundary")
    bcs_ = [bc_]
    u_ = Function(Vplus)
    solve(a_==L_,u_,bcs_)
    print("u inf norm:",np.max(u_.vector()[:]))
    #
    if False:
        L2_err = assemble((u-u_)*(u-u_)*dx)**0.5
        L2_nrm = assemble(u*u*dx)**0.5
        print("u L2 relative error:",L2_err/L2_nrm)
    else:
        # Recommended approach to avoid cancellation errors...
        # marginal difference...
        ui = Function(Vplus)
        ui.interpolate(u)
        ui.vector()[:] -= u_.vector()[:]
        L2_err = assemble(ui*ui*dx)**0.5
        L2_nrm = assemble(u*u*dx)**0.5
        print("u L2 relative error:",L2_err/L2_nrm)
    #
    dudx_ = project(u_.dx(0),Vplus)
    dudy_ = project(u_.dx(1),Vplus)
    L2_err = assemble((dudx-dudx_)*(dudx-dudx_)*dx)**0.5
    L2_nrm = assemble(dudx*dudx*dx)**0.5
    print("dudx L2 relative error:",L2_err/L2_nrm)
    L2_err = assemble((dudy-dudy_)*(dudy-dudy_)*dx)**0.5
    L2_nrm = assemble(dudy*dudy*dx)**0.5
    print("dudy L2 relative error:",L2_err/L2_nrm)

In [ ]:
# Plot the basin in the case of a spherical particle
# Examine the potential in the spherical particle case...
# With a square duct cross-section
U = 10.0
Ufun = lambda y:U*u(y[0],y[1])

y0 = [0.8,0.5]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)

Cs = -0.5*Ufun(y0)+ez0
V_s = lambda y:0.5*(Cs+0.5*Ufun(y))**2
H_s = lambda y,dy:0.5*(dy[0]**2+dy[1]**2)+V_s(y)

# Show the entire potential with bounding lines
if False:
    plt.figure(figsize=(9,4))
    #plt.contourf(X,Y,V_s([X,Y]),17)
    plt.contourf(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,17)
    plt.colorbar()
    if True:
        #plt.contour(X,Y,V_s([X,Y]),levels=[0.5],linestyles=['-'],colors=['k'])
        plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
    plt.gca().set_aspect(1.0)
    plt.show()

# Show just the relevant portion
plt.figure(figsize=(9,4))
#plt.contourf(X,Y,V_s([X,Y]),np.linspace(0,0.5,17))
plt.contourf(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,np.linspace(0,0.5,17))
plt.colorbar()
if True:
    #plt.contour(X,Y,V_s([X,Y]),levels=[0.5],linestyles=['-'],colors=['k'])
    plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
plt.gca().set_aspect(1.0)
plt.show()

# Zoom in on a couple of parts of the relevant portion
if False:
    #plt.contourf(X,Y,V_s([X,Y]),np.linspace(0,0.5,17))
    plt.contourf(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,np.linspace(0,0.5,17))
    plt.colorbar()
    if True:
        #plt.contour(X,Y,V_s([X,Y]),levels=[0.5],linestyles=['-'],colors=['k'])
        plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
    plt.gca().set_aspect(1.0)
    plt.xlim(-0.2,0.2)
    plt.ylim(0.5,0.9)
    plt.show()

    #plt.contourf(X,Y,V_s([X,Y]),np.linspace(0,0.5,17))
    plt.contourf(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,np.linspace(0,0.5,17))
    plt.colorbar()
    if True:
        #plt.contour(X,Y,V_s([X,Y]),levels=[0.5],linestyles=['-'],colors=['k'])
        plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
    plt.gca().set_aspect(1.0)
    plt.xlim(0.4,0.9)
    plt.ylim(0.4,0.9)
    plt.show()

In [ ]:
# Solve the ODE governing the motion of a spherical active particle
ti = 0.0
tf = 2.0**8
ts = np.linspace(ti,tf,1+2**14)
def ode_fun(t,v,Cs):
    x = v[0]
    y = v[1]
    e_x = v[2]
    e_y = v[3]
    e_z = Cs+0.5*Ufun(v[:2])
    result = [e_x,
              e_y,
              -0.5*e_z*U*dudx(x,y),
              -0.5*e_z*U*dudy(x,y),
             ]
    return result

# Example 1

y0 = [0.8,0.5]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)

Cs = -0.5*Ufun(y0)+ez0
V_s = lambda y:0.5*(Cs+0.5*Ufun(y))**2
H_s = lambda y,dy:0.5*(dy[0]**2+dy[1]**2)+V_s(y)

plt.figure(figsize=(9,4))
plt.contourf(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,np.linspace(0,0.5,11))
plt.colorbar()
if False:
    plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
else:
    plt.contour(Xs,Ys,Cs+0.5*U*Us,levels=[-1,1],linestyles=['-','--'],colors=['k'],linewidths=[1])
plt.plot(outline[0],outline[1],'r-',lw=1)
#plt.xticks(np.linspace(-1,1,5))
plt.gca().set_aspect(1.0)
#plt.savefig('Potential_example.eps',**fig_opts)
plt.show()

v0 = y0+dy0
#solution = solve_ivp(ode_fun,[ti,tf],v0,t_eval=ts,args=(Cs,),
#                     method='DOP853',rtol=1.E-9,atol=1.E-12)
integrator = ode(ode_fun).set_integrator('dop853',atol=1.0E-12,rtol=1.0E-9)
integrator.set_initial_value(v0, ts[0]).set_f_params(Cs)
k = 0
solution = np.empty((len(v0),len(ts)))
solution[:,0] = v0
while integrator.successful() and integrator.t < ts[-1]:
    solution[:,k+1] = integrator.integrate(ts[k+1])
    k += 1

plt.figure(figsize=(9,4))
#plt.plot(solution.y[0],solution.y[1],lw=1)
plt.plot(solution[0],solution[1],lw=1)
if False:
    plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
else:
    plt.contour(Xs,Ys,Cs+0.5*U*Us,levels=[-1,1],linestyles=['-','--'],colors=['k'],linewidths=[1])
plt.plot(outline[0],outline[1],'r-',lw=1)
#plt.xticks(np.linspace(-1,1,5))
plt.gca().set_aspect(1.0)
#plt.savefig('Orbit_example.eps',**fig_opts)
plt.show()

plt.figure(figsize=(9,4))
ez = np.array([Cs+0.5*Ufun([solution[0,i],solution[1,i]]) for i in range(len(solution[0]))])
Ham = 0.5*(solution[2]**2+solution[3]**2+ez**2)
plt.plot(ts,Ham,lw=1)
#plt.savefig('Hamiltonian_preservation.eps',**fig_opts)
plt.show()

In [ ]:
# Optionally, try integrating using a RKN method...
# Runge-Kutta-Nystrom solver (a supposed symplectic 4th order method from wiki...)
def RKN_method(ode_fun,y0,dy0,dt,nt,t0=0.,args={}):
    """Here ode_fun is such that \ddot{y}=f(y)"""
    r3 = 3.0**0.5
    c1 = (3.+r3)/6.
    c2 = (3.-r3)/6.
    c3 = c1
    a21 = (2.-r3)/12.
    a32 = r3/6.
    bb1 = (5.-3.*r3)/24.
    bb2 = (3.+r3)/12.
    bb3 = (1.+r3)/24.
    b1 = (3.-2.*r3)/12.
    b2 = 0.5
    b3 = (3.+2.*r3)/12.
    assert len(y0)==len(dy0)
    ys = np.empty((1+nt,len(y0)))
    dys = np.empty((1+nt,len(dy0)))
    ys[0,:] = y0
    dys[0,:] = dy0
    for m in range(nt):
        g1 = ys[m]+c1*dt*dys[m]
        fg1 = ode_fun(g1,**args)
        g2 = ys[m]+dt*(c2*dys[m]+dt*a21*fg1)
        fg2 = ode_fun(g2,**args)
        g3 = ys[m]+dt*(c3*dys[m]+dt*a32*fg2)
        fg3 = ode_fun(g3,**args)
        ys[m+1] = ys[m]+dt*(dys[m]+dt*(bb1*fg1+bb2*fg2+bb3*fg3))
        dys[m+1] = dys[m]+ dt*(b1*fg1+b2*fg2+b3*fg3)
    return ys,dys

if False:
    dudx = project(u.dx(0),V)
    dudy = project(u.dx(1),V)
else:
    # This apears to be slightly worse, even though I expected an improvement...
    V_DG = FunctionSpace(mesh,"DG",degree)
    dudx = project(u.dx(0),V_DG)
    dudy = project(u.dx(1),V_DG)
    
def ode_fun(xy,Cs,U): # leave t out
    x = xy[0]
    y = xy[1]
    e_z = Cs+0.5*Ufun(xy)
    return np.array([-0.5*e_z*U*dudx(x,y),-0.5*e_z*U*dudy(x,y)])

#dt = ts[1]     # 1/64
#nt = len(ts)-1 # 256*64
#dt = 0.5**6
#nt = 2**(8+6)
dt = 0.5**8
nt = 2**(10+8)
ts = np.linspace(0,dt*nt,nt+1)
ys,dys = RKN_method(ode_fun,y0,dy0,dt,nt,args={'Cs':Cs,'U':U})

plt.figure(figsize=(9,4))
plt.plot(ys[:,0],ys[:,1],lw=1)
if False:
    plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
else:
    plt.contour(Xs,Ys,Cs+0.5*U*Us,levels=[-1,1],linestyles=['-','--'],colors=['k'],linewidths=[1])
plt.plot(outline[0],outline[1],'r-',lw=1)
plt.gca().set_aspect(1.0)
#plt.savefig('Orbit_example.eps',**fig_opts)
plt.show()

plt.figure(figsize=(9,4))
ez = np.array([Cs+0.5*Ufun([ys[i,0],ys[i,1]]) for i in range(len(ys[:,0]))])
Ham = 0.5*(dys[:,0]**2+dys[:,1]**2+ez**2)
plt.plot(ts,Ham-0.5,lw=1)
#plt.savefig('Hamiltonian_preservation.eps',**fig_opts)
plt.show()

In [ ]:
# Produce a curvilinear meshgrid sampling
xs = np.linspace(-W/2,W/2,nx+1)
ys = np.linspace(-H/2,H/2,ny+1)
Xs,Ys = np.meshgrid(xs,ys,indexing='ij')
Ys *= (1-sp*np.cos(Xs*np.pi/W)**2)
Us = np.empty_like(Xs)
#u.set_allow_extrapolation(False)
for i in range(nx+1):
    x = Xs[i,j]
    for j in range(ny+1):
        y = Ys[i,j]
        Us[i,j] = u(x,y)
from scipy.interpolate import RectBivariateSpline as RBS
U_RBS = RBS(xs,ys,Us)
plt.contourf(Xs,Ys,U_RBS(xs,ys),17)
#plt.contourf(Xs,Ys,U_RBS.ev(Xs,Ys),17) # <-- this isn't right, need to map back to reference grid
plt.colorbar()
plt.gca().set_aspect(1.0)
plt.show()

# Try to solve the ODE using the RBS
pi_W = np.pi/W
def ode_fun(xy,Cs,U): # leave t out
    # optimised
    x = xy[0]
    c = (1-sp*np.cos(x*np.pi/W)**2)
    y = xy[1]/c
    e_z = Cs+0.5*U*U_RBS.ev(x,y)
    U_x = U_RBS.ev(x,y,dx=1)*U
    U_y = U_RBS.ev(x,y,dy=1)*U/c
    return np.array([-0.5*e_z*(U_x-y*U_y*pi_W*sp*np.sin(2*pi_W*x)),
                     -0.5*e_z*U_y])

dt = 0.5**8
nt = 2**(10+8)
ts = np.linspace(0,dt*nt,nt+1)
Cs = -0.5*U*U_RBS.ev(y0[0],y0[1]/(1-sp*np.cos(y0[0]*np.pi/W)**2))+ez0
ys,dys = RKN_method(ode_fun,y0,dy0,dt,nt,args={'Cs':Cs,'U':U})

plt.figure(figsize=(9,4))
plt.plot(ys[:,0],ys[:,1],lw=1)
if False:
    plt.contour(Xs,Ys,0.5*(Cs+0.5*U*Us)**2,levels=[0.5],linestyles=['-'],colors=['k'])
else:
    plt.contour(Xs,Ys,Cs+0.5*U*Us,levels=[-1,1],linestyles=['-','--'],colors=['k'],linewidths=[1])
plt.plot(outline[0],outline[1],'r-',lw=1)
plt.gca().set_aspect(1.0)
#plt.savefig('Orbit_example.eps',**fig_opts)
plt.show()

plt.figure(figsize=(9,4))
#ez = np.array([Cs+0.5*Ufun([ys[i,0],ys[i,1]]) for i in range(len(ys[:,0]))])
ez = np.array([Cs+0.5*U*U_RBS.ev(ys[i,0],ys[i,1]/(1-sp*np.cos(ys[i,0]*np.pi/W)**2)) for i in range(len(ys[:,0]))])
Ham = 0.5*(dys[:,0]**2+dys[:,1]**2+ez**2)
plt.plot(ts,Ham-0.5,lw=1)
#plt.savefig('Hamiltonian_preservation.eps',**fig_opts)
plt.show()